In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [9]:
df=pd.read_csv('C:/Users/Veera/Downloads/diabetes.csv')

In [10]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [5]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [6]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [11]:
x=df.drop(columns='Outcome')
y=df['Outcome']

## Feature selection

In [13]:
from feature_engine.selection import DropConstantFeatures,DropCorrelatedFeatures,DropDuplicateFeatures
from sklearn.pipeline import Pipeline
x=df.drop(columns='Outcome')
pip=Pipeline([
              ('Constant',DropConstantFeatures(tol=0.99)),
              ('Duplicate',DropDuplicateFeatures())
])
x_new=pip.fit_transform(x)

In [14]:
from feature_engine.selection import SmartCorrelatedSelection
sel=SmartCorrelatedSelection(selection_method='variance')
x_new=sel.fit_transform(x)

In [15]:
x_new.shape

(768, 8)

### Applying automl

In [16]:
import evalml

In [17]:
from evalml.preprocessing import split_data
xtrain,xtest,ytrain,ytest=split_data(x_new,y,problem_type='binary')

In [18]:
from evalml.automl import AutoMLSearch

In [22]:
best=AutoMLSearch(xtrain,ytrain,problem_type='binary')
best.search()

	High coefficient of variation (cv >= 0.5) within cross validation scores.
	Decision Tree Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Select Columns Transformer may not perform as estimated on unseen data.


{1: {'Logistic Regression Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Standard Scaler': '00:03',
  'Random Forest Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer': '00:01',
  'Total time of batch': '00:04'},
 2: {'Logistic Regression Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Standard Scaler + RF Classifier Select From Model': '00:02',
  'Random Forest Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + RF Classifier Select From Model': '00:01',
  'Total time of batch': '00:04'},
 3: {'Decision Tree Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Select Columns Transformer': '00:01',
  'LightGBM Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Select Columns Transformer': '00:02',
  'Extra Trees Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Select Columns Transformer': '00:01',
  'E

In [23]:
best.rankings

,id,pipeline_name,search_order,validation_score,mean_cv_score,standard_deviation_cv_score,percent_better_than_baseline,high_variance_cv,parameters
0,7,Extra Trees Classifier w/ Label Encoder + Repl...,7,0.504895,0.504895,0.008812,95.805797,False,"{'Label Encoder': {'positive_label': None}, 'I..."
1,4,Random Forest Classifier w/ Label Encoder + Re...,4,0.505822,0.505822,0.014101,95.798091,False,"{'Label Encoder': {'positive_label': None}, 'I..."
2,8,Elastic Net Classifier w/ Label Encoder + Repl...,8,0.509015,0.509015,0.015441,95.771566,False,"{'Label Encoder': {'positive_label': None}, 'I..."
3,2,Random Forest Classifier w/ Label Encoder + Re...,2,0.510350,0.510350,0.011367,95.760475,False,"{'Label Encoder': {'positive_label': None}, 'I..."
4,3,Logistic Regression Classifier w/ Label Encode...,3,0.513611,0.513611,0.008336,95.733394,False,"{'Label Encoder': {'positive_label': None}, 'I..."
5,1,Logistic Regression Classifier w/ Label Encode...,1,0.514688,0.514688,0.012160,95.724441,False,"{'Label Encoder': {'positive_label': None}, 'I..."
6,9,CatBoost Classifier w/ Label Encoder + Replace...,9,0.643033,0.643033,0.000256,94.658274,False,"{'Label Encoder': {'positive_label': None}, 'I..."
7,10,XGBoost Classifier w/ Label Encoder + Replace ...,10,0.656306,0.656306,0.017778,94.548012,False,"{'Label Encoder': {'positive_label': None}, 'I..."
8,6,LightGBM Classifier w/ Label Encoder + Replace...,6,0.672722,0.672722,0.034657,94.411639,False,"{'Label Encoder': {'positive_label': None}, 'I..."
9,5,Decision Tree Classifier w/ Label Encoder + Re...,5,4.564685,4.564685,1.318751,62.080771,True,"{'Label Encoder': {'positive_label': None}, 'I..."


In [24]:
pipe=best.best_pipeline
pipe

pipeline = BinaryClassificationPipeline(component_graph={'Label Encoder': ['Label Encoder', 'X', 'y'], 'Replace Nullable Types Transformer': ['Replace Nullable Types Transformer', 'X', 'Label Encoder.y'], 'Imputer': ['Imputer', 'Replace Nullable Types Transformer.x', 'Replace Nullable Types Transformer.y'], 'Select Columns Transformer': ['Select Columns Transformer', 'Imputer.x', 'Replace Nullable Types Transformer.y'], 'Extra Trees Classifier': ['Extra Trees Classifier', 'Select Columns Transformer.x', 'Replace Nullable Types Transformer.y']}, parameters={'Label Encoder':{'positive_label': None}, 'Imputer':{'categorical_impute_strategy': 'most_frequent', 'numeric_impute_strategy': 'mean', 'boolean_impute_strategy': 'most_frequent', 'categorical_fill_value': None, 'numeric_fill_value': None, 'boolean_fill_value': None}, 'Select Columns Transformer':{'columns': ['Glucose', 'BMI', 'DiabetesPedigreeFunction', 'Age']}, 'Extra Trees Classifier':{'n_estimators': 100, 'max_features': 'auto', 

##  Best classifer:
#### Extra Trees Classifier w/ Label Encoder 

## Best Pipeline:
#### Extra Trees Classifier w/ Label Encoder + Replace Nullable Types Transformer + Imputer + Select Columns Transformer